# Proofread_nuclei

In [1]:
# libraries 1
import numpy as np
import pyperclip
import pandas as pd
import pyperclip
from cloudvolume import CloudVolume, view, Bbox
from nglui import statebuilder,annotation,easyviewer,parser
from nglui.statebuilder import *
import json

import sys
import os
from datetime import datetime
from caveclient import CAVEclient

sys.path.append(os.path.abspath("../segmentation"))
import authentication_utils as auth
import rootID_lookup as IDlook
sys.path.append(os.path.abspath("../synapses"))
import connectivity_utils

In [2]:
datastack_name = 'fanc_production_mar2021'
client = CAVEclient(datastack_name)

## Create links for proofreading nuclei/somas

Good example: https://github.com/seung-lab/NeuroglancerAnnotationUI/blob/master/examples/statebuilder_examples.ipynb

In [3]:
df = pd.read_csv('../Output/soma_info_Aug2021ver4vol.csv', header=0)

In [4]:
# we only need xyz for nuclei/somas..., and make them into lines
xyz_df = df.reindex(columns=['nuc_xyz', 'soma_xyz'])
nuc_xyz_df = df['nuc_xyz'].str.strip('()').str.split(',',expand=True)
soma_xyz_df = df['soma_xyz'].str.strip('()').str.split(',',expand=True)
xyz_df['nuc_xyz'] = nuc_xyz_df.astype(int).values.tolist()
xyz_df['soma_xyz'] = soma_xyz_df.astype(int).values.tolist()

In [5]:
nuc_xyz_df = nuc_xyz_df.set_axis(['Col_x', 'Col_y', 'Col_z'], axis=1)
sorted_indices = nuc_xyz_df.astype(int).sort_values(by=['Col_y'], ascending=True).index
xyz_df = xyz_df.reindex(sorted_indices).reset_index(drop=True)

In [6]:
# ..., and make them into lines
lines = LineMapper(point_column_a='nuc_xyz', point_column_b='soma_xyz')

In [7]:
# make Neuroglancer link
imgTokyo = ImageLayerConfig(name = 'FANCv4-jp',
                                    source = auth.get_cv_path('Image_Tokyo')['url'])
img = ImageLayerConfig(name = 'FANCv4',
                                    source = auth.get_cv_path('Image')['url'])
seg = SegmentationLayerConfig(name = 'seg_Mar2021_proofreading',
                                    source = auth.get_cv_path('FANC_production_segmentation')['url'])     
nuc_Aug = ImageLayerConfig(name = 'nuc',
                                    source = auth.get_cv_path('nuclei_map_Aug2021')['url'])      
nuc_seg_Aug = ImageLayerConfig(name = 'nuc_seg',
                                    source = auth.get_cv_path('nuclei_seg_Aug2021')['url'])        

ann = AnnotationLayerConfig(name='nuc_soma_Aug2021',
                            mapping_rules=lines,
                            tags=['neuron', 'glia','false_positive'],
                            active = True)

In [26]:
view_options = {'layout': 'xy'}

In [29]:
sb = StateBuilder(layers=[imgTokyo, seg, img, nuc_Aug, nuc_seg_Aug, ann],
                  resolution=[4.3,4.3,45],
                  view_kws=view_options)
# viewerState
# vs = {'gpuMemoryLimit': 4000000000,
#       'systemMemoryLimit': 4000000000,
#       'concurrentDownloads': 64}

LINK=[]
k=500
minidfs = [xyz_df.loc[i:i+k-1, :] for i in range(0, len(xyz_df), k)]
for dftmp in minidfs:
    # csb = ChainedStateBuilder([sb, vs])                             
    state = json.loads(sb.render_state(dftmp, return_as='json'))
    jsn_id = client.state.upload_state_json(state)
    output = client.state.build_neuroglancer_url(jsn_id, auth.get_cv_path('neuroglancer_base')['url'])
    # output =auth.get_cv_path('neuroglancer_base')['url'] + '?json_url={path}{state_id}'.format(path=auth.get_cv_path('json_server')['url'],state_id=client.state.upload_state_json(state))
    LINK.append(output)

In [30]:
# save into csv
LINK2 = pd.DataFrame(LINK)
LINK2.to_csv('../Output/links_20210902.csv', index=False, header=False)

In [ ]:
import numpy as np
import sys
import os
import pandas as pd
from glob import glob
import argparse
import  sqlite3

from cloudvolume import CloudVolume, Bbox
import fill_voids
from taskqueue import TaskQueue, queueable, LocalTaskQueue
from functools import partial
from lib import *
sys.path.append(os.path.abspath("../segmentation"))
# to import rootID_lookup and authentication_utils like below
import rootID_lookup as IDlook
import authentication_utils as auth

In [ ]:

# variables
np.random.seed(123)
window_coef = 4 # window size to get nuclei in mip2
output_name = 'soma_info_Aug2021ver4vol'

# could-volume url setting
seg = CloudVolume(auth.get_cv_path('FANC_production_segmentation')['url'], use_https=True, agglomerate=False, cache=True, progress=False) # mip2
nuclei_seg_cv = CloudVolume(auth.get_cv_path('nuclei_seg_Aug2021')['url'], cache=False, progress=False, use_https=True,autocrop=True, bounded=False) # mip4
# read csv

# -----------------------------------------------------------------------------


def vol_shift(input, pixel): # this is still very slow since this overuse RAM even though np.roll is fast
  # x plane
  x_p = np.roll(input, pixel, axis=0)
  x_p[:pixel,:,:] = 0
  x_n = np.roll(input, -pixel, axis=0)
  x_n[-pixel:,:,:] = 0
  # y plane
  y_p = np.roll(input, pixel, axis=1)
  y_p[:,:pixel,:] = 0
  y_n = np.roll(input, -pixel, axis=1)
  y_n[:,-pixel:,:] = 0
  # z plane
  z_p = np.roll(input, pixel, axis=2)
  z_p[:,:,:pixel] = 0
  z_n = np.roll(input, -pixel, axis=2)
  z_n[:,:,-pixel:] = 0

  sum = x_p + x_n + y_p + y_n + z_p + z_n
  result = sum - input*6
  result[result>0] = 1
  result[result<0] = 0

  result = result.astype('int64')

  return result


def argwhere_from_outside(volume, value, bbox_size):
  ind = np.argwhere(volume == value)
  center = bbox_size/2

  distance = np.apply_along_axis(np.linalg.norm, 1, ind-center)
  sorted_indice = np.argsort(-distance)
  result = ind[sorted_indice]
  
  return result

In [ ]:
rowi = df.iloc[21,:].values

In [ ]:
rowi

In [ ]:

cord_mip0 = rowi[1:4]
cord_mip4 = mip0_to_mip4_array(cord_mip0)
bbox_size = np.array([rowi[15]*window_coef, rowi[16]*window_coef, rowi[17]*window_coef], dtype='int64')

seg_nuc = nuclei_seg_cv.download_point(pt=cord_mip4, size=bbox_size, mip=[68.8,68.8,45.0]) #mip4

vol_temp = seg_nuc[:,:,:]
vol_temp2 = np.where(vol_temp == rowi[11], 1, 0)
vol = np.squeeze(vol_temp2)
voxel_size = np.count_nonzero(vol == 1)

In [ ]:


filled = fill_voids.fill(vol, in_place=False) # fill the empty space with fill_voids. Ignore DeprecationWarning
shifted = vol_shift(filled, pixel=1) # shift the volume one pixel

location_one = argwhere_from_outside(shifted, value=1, bbox_size=bbox_size)

In [ ]:
ind = np.argwhere(shifted == 1)
center = bbox_size/2

distance = np.apply_along_axis(np.linalg.norm, 1, ind-center)

In [ ]:
distance

In [ ]:
sorted_indice = np.argsort(-distance)
result = ind[sorted_indice]

In [ ]:
sorted_indice

In [ ]:
ind[36]

In [ ]:
result